# <span style='font-family: CMU Sans Serif, sans-serif;'> How to retrieve data from Global Factor Data  </span> 

The database is hosted on Wharton Research Data Service (WRDS) and developed by Theis I. Jensen (Yale) Bryan Kelly (Yale, AQR Capital, and NBER), and Lasse H. Pedersen (Copenhagen Business School and AQR Capital). 

This code shows how to connect to that database and query data from it.

## <span style='font-family: CMU Sans Serif, sans-serif;'> Preamble  </span> 

### <span style='font-family: CMU Sans Serif, sans-serif;'> Package import  </span> 

First the necessary libraries are imported (`dotenv` is not strictly necessary but good for hiding personal information such as usernames and passwords).

In [1]:
from dotenv import load_dotenv

import pandas as pd
import wrds
import os

### <span style='font-family: CMU Sans Serif, sans-serif;'> Notebook setup  </span> 

In [2]:
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', 0)  # Use full cell width
pd.set_option('display.expand_frame_repr', False)  # Prevent line breaks

### <span style='font-family: CMU Sans Serif, sans-serif;'> Credentials import  </span> 

Password and user-name to Wharton Research Data Services are loaded into environment with `.env` file extension to keep these credentials hidden.

In [3]:
load_dotenv()
wrds_usr = os.getenv("MY_WRDS_USERNAME")
wrds_pw = os.getenv("MY_WRDS_PASSWORD")

## <span style='font-family: CMU Sans Serif, sans-serif;'> Data import  </span> 

Below a `SQL` query is created and data is queried from the `contrib.global_factor` database hosted on WRDS.

### <span style='font-family: CMU Sans Serif, sans-serif;'> Initialize connection to WRDS   </span> 

The code below connects this session the WRDS server, so that *all* data available on the website can be pulled into this session via en `SQL` query.

In [4]:
wrds_db = wrds.Connection(
    wrds_username = wrds_usr, 
    wrds_password = wrds_pw)

Loading library list...
Done


### <span style='font-family: CMU Sans Serif, sans-serif;'> Available variables  </span> 

The code below creates lists of the available variables in the database `contrib.global_factor`.

In [5]:
# Import excel sheets from official GitHub page (created by Global Factor Data team)
countries = pd.read_excel('https://github.com/bkelly-lab/ReplicationCrisis/raw/master/GlobalFactors/Country%20Classification.xlsx')
variables = pd.read_excel('https://github.com/bkelly-lab/ReplicationCrisis/raw/master/GlobalFactors/Factor%20Details.xlsx')

The dataframes of available data are cleaned and corrected in the code below. Notice we only select developed or emerging markets.

In [6]:
countries = countries[countries['msci_development'].isin(('developed', 'emerging'))][['excntry', 'msci_development']]
variables = variables[variables['abr_jkp'].notna()][['abr_jkp', 'name_new']]

The variables we can select are shown below.

In [7]:
print(variables.T)
print("")
print(countries.T)

                                     0                          3                           4                              6                     7                                                  9                                           10                            11                             12                     19              22                24                  26                 27                       30              32            34                36                      38                     39                                 40               42                             44                         47                               49                 51               53                             56            57                        59                    60                              61                                        62                     63                      64                      65                66                 68                           

### <span style='font-family: CMU Sans Serif, sans-serif;'> Querying data from WRDS   </span> 

This is an example query that pulls data from the previously talked about database. The SQL used for this can be found in the `query.sql` file in this folder.

In [9]:
with open("query.sql", "r") as file: 
    sql_query = file.read()

data = wrds_db.raw_sql(sql_query)

/Users/albertschroder/Desktop/BACHELOR NOTES/bachelor_venv/lib/python3.11/site-packages/wrds/sql.py:591: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  full_df = pd.concat([full_df, chunk])


To avoid running this again we save the data as a compressed csv file.

In [12]:
data.to_parquet("usa__gfd.parquet", engine="pyarrow", compression="snappy")

In [13]:
pd.set_option('display.max_rows', None)  # Show all rows
print(pd.DataFrame(data.isna().sum()))

                            0
capex_abn             1775268
z_score               1322279
ami_126d               825114
at_gr1                 863896
tangibility           1144876
sale_bev               899889
at_me                  748812
at_be                  843501
bev_mev                955783
be_me                  881272
capx_gr1              1259420
capx_gr2              1485466
capx_gr3              1707909
at_turnover            787078
ocfq_saleq_std        2063979
cop_at                1080503
cop_atl1              1109823
cash_at                749502
dgp_dsale             1358198
be_gr1a               1062948
coa_gr1a              1208691
col_gr1a              1170604
cowc_gr1a             1232328
fnl_gr1a               879421
lti_gr1a              1148751
lnoa_gr1a             1360426
nfna_gr1a              879421
nncoa_gr1a            1237515
noa_gr1a              1260095
ncoa_gr1a             1186376
ncol_gr1a             1198150
ocf_at_chg1            984675
niq_at_chg

In [ ]:
data.shape

(4211606, 192)

: 